# Pokerole SRD Builder

You don't need to use this, if you're looking for the Obsidian SRD, it's [here](https://github.com/Willowlark/PokeroleObsidianSRD).

In [2]:
import yaml
from glob import glob
import json
import os
import pandas as pd
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from shutil import copy

secrets = json.load(open('../secrets.json'))

pokedex_path = '../Version20/Pokedex'
abilities_path = '../Version20/Abilities'
moves_path = '../Version20/Moves'
learnsets_path = '../Version20/Learnsets'
natures_path = '../Version20/Natures'
sprites_path = '../Images/BoxSprites/'
home_path = '../Images/HomeSprites/'
book_path = '../Images/BookSprites/'
items_path = '../Version20/Items/'

pokedex_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Pokedex/'
abilities_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Abilities/'
moves_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Moves/'
learnsets_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Learnsets/'
natures_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Natures/'
sprites_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-BoxSprites/'
home_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-HomeSprites/'
book_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-BookSprites/'
items_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Items/'

os.makedirs(pokedex_output,exist_ok=True)
os.makedirs(abilities_output,exist_ok=True)
os.makedirs(moves_output,exist_ok=True)
os.makedirs(learnsets_output,exist_ok=True)
os.makedirs(natures_output,exist_ok=True)
os.makedirs(sprites_output,exist_ok=True)
os.makedirs(home_output,exist_ok=True)
os.makedirs(book_output,exist_ok=True)
os.makedirs(items_output,exist_ok=True)

/Users/bill/opt/miniconda3/envs/sink/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [1]:
ability_template = '''## `= this.name`

> *`= this.Description`*

**Effect:** `= this.Effect`'''

#-------------------------------------------------------------

moves_template = '''### `= this.name` 
*`= this.Description`*

**Accuracy:** `= this.Accuracy1` + `= this.Accuracy2`
**Damage:** `= this.Power` `= choice(length(this.Damage1)=0, "","\+ "+ this.Damage1)` `= choice(length(this.Damage2)=0, "","\+ "+ this.Damage2)`

| Type          | Target          | Damage Type          | Power          |
| ------------- | --------------- | ---------------- | -------------- |
| `= this.Type` | `= this.Target` | `= this.DmgType` | `= this.Power` | 

**Effect:** `= this.Effect`'''

#-------------------------------------------------------------

learnsets_template = '''## `= this.Name` Learnset

**DexID:** `= this.DexID`

```dataview
TABLE WITHOUT ID
    T[0] AS Learned,
    T[1] AS Move
FROM #PokeroleSRD/Learnsets
flatten moves as T
where file.path = this.file.path
```
'''

#-------------------------------------------------------------------

natures_template = '''## `= this.Nature`

**Confidence**: `= this.Confidence`

*`= this.Keywords`*

> `= this.Description`
'''

#-------------------------------------------------------------------

items_template = '''## `= this.Name`

*`= this.Description`*

| Type Bonus         | Value          | Heal Amount         | Suggested Price         | PMD Price         |
| ------------------ | -------------- | ------------------- | ----------------------- | ----------------- |
| `= this.TypeBonus` | `= this.Value` | `= this.HealAmount` | `= this.SuggestedPrice` | `= this.PMDPrice` |

**Pokemon Limitation**: `= this.SpecificPokemon`
'''

# Pokedex Entries

In [7]:
for src in glob(pokedex_path+"/*.json"):
    entry = json.loads(open(src).read())
    sname = entry['Sprite'].split('.')
    entry['BoxSprite'] = f"SRD-{sname[0]}-BoxSprite.{sname[1]}"
    entry['HomeSprite'] = f"SRD-{sname[0]}-HomeSprite.{sname[1]}"
    entry['BookSprite'] = f"SRD-{sname[0]}-BookSprite.{sname[1]}"
    
    entry['Unevolved'] = 'Yes' if entry['Unevolved'] else 'No'
    entry['HasForm'] = 'Yes' if entry['HasForm'] else 'No'
    entry['Legendary'] = 'Yes' if entry['Legendary'] else 'No'
    entry['GoodStarter'] = 'Yes' if entry['GoodStarter'] else 'No'
    entry['Baby'] = 'Yes' if entry['Baby'] else 'No'
    entry['Learnset'] = f"[[SRD-{entry['Name']}-Learnset]]"
    
    if 'MegaEvolutions' in entry:
        mega = "\n"
        for x in entry['MegaEvolutions']:
            mega = mega+f"**MegaEvolution**:: [[SRD-{x['Name']}]]\nvia [[SRD-{x['Item']}]]\n"
    elif 'MegaBaseForm' in entry:
        mega = f"\n**MegaBaseForm**:: [[SRD-{entry['MegaBaseForm']}]]\n"
        del entry['MegaBaseForm']
    else:
        mega = ""
    
    abilities = (f"[[SRD-{entry['Ability1']}|{entry['Ability1']}]]"
                        f"{'' if not entry['Ability2'] else ' / [[SRD-'+ entry['Ability2']+'|'+entry['Ability2']+']]'}"
                        f"{'' if not entry['HiddenAbility'] else ' ([[SRD-'+entry['HiddenAbility']+'|'+entry['HiddenAbility']+']])'}"
                        f"{'' if not entry['EventAbilities'] else ' <[[SRD-'+entry['EventAbilities']+'|'+entry['EventAbilities']+']]>'}"
                        )
    INTEGERS = ['BaseHP', 'Strength', 'MaxStrength',
       'Dexterity', 'MaxDexterity', 'Vitality', 'MaxVitality', 'Special',
       'MaxSpecial', 'Insight', 'MaxInsight']
    for key in INTEGERS:
        entry[key] = int(entry[key])

    template = f"""# `= this.name`

![[{entry['BookSprite']}|right wsmall]]

*{entry['DexCategory']}*
*{entry['DexDescription']}*

**DexID**:: {entry['DexID']}
**Species**:: {entry['Name']}
**Type**:: {entry['Type']}
**Abilities**:: {abilities}
**Base HP**:: {entry['BaseHP']}

|           |                                                                                        |                                          |
| --------- | -------------------------------------------------------------------------------------- | ---------------------------------------- |
| Strength  | `= padleft(padright("",this.MaxStrength - this.Strength,"⭘"),this.MaxStrength,"⬤")`    | (Strength::{entry['Strength']})/(MaxStrength::{entry['MaxStrength']})   |
| Dexterity | `= padleft(padright("",this.MaxDexterity - this.Dexterity,"⭘"),this.MaxDexterity,"⬤")` | (Dexterity:: {entry['Dexterity']})/(MaxDexterity::{entry['MaxDexterity']}) |
| Vitality  | `= padleft(padright("",this.MaxVitality - this.Vitality,"⭘"),this.MaxVitality,"⬤")`    | (Vitality::{entry['Vitality']})/(MaxVitality::{entry['MaxVitality']})   |
| Special   | `= padleft(padright("",this.MaxSpecial - this.Special,"⭘"),this.MaxSpecial,"⬤")`       | (Special::{entry['Special']})/(MaxSpecial::{entry['MaxSpecial']})     |
| Insight   | `= padleft(padright("",this.MaxInsight - this.Insight,"⭘"),this.MaxInsight,"⬤")`       | (Insight::{entry['Insight']})/(MaxInsight::{entry['MaxInsight']})     |

![[{entry['HomeSprite']}|right]]
![[{entry['BoxSprite']}|right]]

**Recommended Rank**:: {entry['RecommendedRank']}
**Good Starter**:: {entry['GoodStarter']}
**Can Evolve**:: {entry['Unevolved']}{mega}
**Height**: {str(entry['Height']['Feet']).split('.')[0]}'{str(entry['Height']['Feet']).split('.')[1]}" / {entry['Height']['Meters']}m
**Weight**: {entry['Weight']['Pounds']}lbs / {entry['Weight']['Kilograms']}kg

![[SRD-{entry['Name']}-Learnset]]"""
        
    for x in ['DexID','Strength','MaxStrength','Dexterity','MaxDexterity',
              'Vitality','MaxVitality','Special','MaxSpecial','Insight','MaxInsight',
             'BaseHP', 'RecommendedRank', 'GoodStarter', 'Unevolved', '_id', "Abilities", "Type"]:
        del entry[x]
    
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Pokedex\n\n{template}"
    open(pokedex_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Ability Entries

In [ ]:
for src in glob(abilities_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Abilities\n\n{ability_template}"
    open(abilities_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Move Dex



In [ ]:
for src in glob(moves_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Moves\n\n{moves_template}"
    open(moves_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Move Lists

In [ ]:
for src in glob(learnsets_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    
    entry['Species'] = f"[[SRD-{entry['Name']}|{entry['Name']}]]"
    moves = []
    for m in entry["Moves"]:
        if moves and m["Learned"] != moves[-1][0]:
            moves.append(["---------------------------","---------------------------"])
        moves.append([m[f'Learned'],f'[[SRD-{m["Name"]}|{m["Name"]}]]'])
    entry['Moves'] = moves
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Learnsets\n\n{learnsets_template}"
    open(learnsets_output+f"SRD-{entry['Name']}-Learnset.md",'w').write(entry_output)
    

# Natures

In [ ]:
for src in glob(natures_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Natures\n\n{natures_template}"
    open(natures_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Images

In [ ]:
def x(path, output, postfix):
    for img in [x for x in os.listdir(path) if '.png' in x]:
        sname = img.split('.')
        srdname = f'SRD-{sname[0]}-{postfix}.{sname[1]}'
        copy(path+img, output+srdname)
        # print(path+img, output+srdname)
x(sprites_path, sprites_output, 'BoxSprite')
x(home_path, home_output, 'HomeSprite')
x(book_path, book_output, 'BookSprite')

# Items

In [ ]:
for src in glob(items_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Items\n\n{items_template}"
    open(items_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)